On va tenter de simuler une pseudo carte scolaire. En effet, notre modélisation du lien entre taux de réussite au baccalauréat et revenu souffre probablement du fait qu'on ait associé le revenu des élèves à celui des communes de leur lycée. 
Or, des élèves viennent d'autres communes non pourvues d'établissement scolaire et le revenu médian peut être très hétérogène entre deux communes. 
On va donc essayer de refaire une carte scolaire de la manière suivante :
- Chaque commune sera associée à un point géographique. Si elle est dépourvue de lycée, on l'associera au lycée le plus proche. 
- On ponderera alors un revenu médian pour chaque lycée en fonction du revenu médian de chaque commune qui lui est associée et de la population des communes prises en compte. On posera donc l'hypothèse que la proportion de bacheliers potentiels est identique dans chaque commune (ce qui est discutable étant donné qu'il y'a une ségrégation spatiale générationnelle sur le territoire français)
- On procédera ainsi à une nouvelle régression avec les données obtenues

On doit donc créer une base de données adaptée. Il faut qu'on ait dedans
- Chaque établissement scolaire, avec son nombre de candidats et le taux de réussite par filières. 
- Chaque commune avec le revenu médian associé
- Les coordonnées géographiques des lycées et des communes 
En plus des bases de données utilisées précédemment, nous avons utilisé une base de données de La Poste pour avoir la longitude et la latitude de chaque commune. 

In [1]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git
import requests
url = 'https://github.com/InseeFrLab/Py-Insee-Data/archive/refs/heads/master.zip'
r = requests.get(url)
with open("pynsee.zip" , 'wb') as zipfile:
    zipfile.write(r.content)
!pip install --ignore-installed pynsee.zip
!pip install python-Levenshtein
!pip install openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee.download
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.0 MB/s eta 0:00:00
  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-7t9kljzq
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-7t9kljzq
  Resolved https://github.com/InseeFrLab/Py-Insee-Data.git to commit 916d8bc82ea43f637047ed989fb7dbc491f56895
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 9.5 MB/s eta 0:00:00
  Created wheel for pynsee: filename=pynsee-0.1.2-py3-none-any.whl size=9805198 sha256=a0d9c270dd8a7332d997c84a956445c044d36531905f28f594e1ea15ff915f0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-7c

In [95]:
#Importations des bases de données

#Base de communes
df_communes = pd.read_excel("FILO2019_DEC_COM.xlsx", sheet_name = 1, header=[4,5])
df_com = df_communes.copy()
df_com = df_com.drop(['PMIMP19', 'Q119', 'Q319','Q3_Q1','D119', 'D219', 'D319', 'D419', 'D619', 'D719', 'D819', 'D919', 'RD', 'S80S2019', 'GI19', 'PACT19', 'PTSA19', 'PCHO19', 'PBEN19', 'PPEN19', 'PAUT19'], axis = 1, level = 1 )
#Base des lycées
dflyc = pd.read_excel('base_def_SansDOMTOM.xlsx')
dflyc = dflyc.drop([dflyc["Code_commune"].str.startswith('2B')]==True)
dflyc = dflyc.drop([dflyc["Code_commune"].str.startswith('2A')]==True)

#Base des coordonnées géographiques des communes 
dfgeo = pd.read_csv('communes-departement-region.csv')

#Base population
dfpop = pd.read_excel('ensemble.xlsx', sheet_name=4, header=7)
dfpop =dfpop[["Code commune", "Population totale"]]

KeyError: '[False] not found in axis'

In [90]:
dflyc.columns

Index(['Unnamed: 0', 'Identifiant_de_l_etablissement',
       'Informations établissement_Académie',
       'Informations établissement_Département',
       'Informations établissement_Etablissement',
       'Informations établissement_Ville', 'Code_commune',
       'Informations établissement_Secteur',
       'Nombre d'élèves présents au Bac_GNLE', 'Taux de réussite bruts_GNLE',
       'Valeur ajoutée du taux de réussite_GNLE',
       'Nombre d'élèves présents au Bac_Somme Techno',
       'Taux de réussite bruts_Taux brut techno',
       'Valeur ajoutée du taux de réussite_Valeur ajoutée techno',
       'Nombre d'élèves présents au Bac_TOTAL_LGT',
       'Taux de réussite bruts_TOTAL_LGT',
       'Valeur ajoutée du taux de réussite_TOTAL_LGT',
       'Nombre d'élèves présents au Bac_TOTAL_Pro',
       'Taux de réussite bruts_TOTAL_Pro',
       'Valeur ajoutée du taux de réussite_TOTAL_Pro',
       'Nbre de ménages fiscaux_NBMEN19',
       'Nbre de personnes dans les ménages fiscaux_NB

In [ ]:
### NettoyageBaseGéo
dfgeo=dfgeo[dfgeo['code_postal']<97000]
dfgeo = dfgeo.drop(["code_postal", "libelle_acheminement", "ligne_5", "nom_commune_complet", "nom_departement", "nom_region","article"], axis=1)
dfgeo

#NettoyageBaseLycCom
dflyc.columns
dflyc = dflyc.drop(["Unnamed: 0", "Valeur ajoutée du taux de réussite_GNLE", "Valeur ajoutée du taux de réussite_Valeur ajoutée techno", "Nombre d'élèves présents au Bac_TOTAL_LGT","Taux de réussite bruts_TOTAL_LGT", "Valeur ajoutée du taux de réussite_TOTAL_LGT", "Valeur ajoutée du taux de réussite_TOTAL_Pro", "Nbre de ménages fiscaux_NBMEN19","Nbre de personnes dans les ménages fiscaux_NBPERS19", "Nbre d'unités de consommation dans les ménages fiscaux_NBUC19", "coordonnee_X", "coordonnee_Y", "epsg"], axis=1)
dflyc

#Amélioration base communes
df_com.columns = df_com.columns.map('_'.join).str.strip('_')
df_com = df_com.drop(["Nbre de ménages fiscaux_NBMEN19","Nbre de personnes dans les ménages fiscaux_NBPERS19","Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"], axis=1)
df_com = df_com.drop([df_com["Code géographique_CODGEO"].str.startswith('2A')])
df_com = df_com.drop([df_com["Code géographique_CODGEO"].str.startswith('2B')]) #On enlève la Corse pour faciliter des manipulations futures
df_com["Code géographique_CODGEO"] = df_com["Code géographique_CODGEO"].astype(int)
df_com2 = df_com.merge(dfpop, how='left', left_on = "Code géographique_CODGEO", right_on='Code commune')
df_com2

In [69]:
#Fusion des bases

#Fusion commune/géographie
dfcomgeo  = df_com2.merge(dfgeo, left_on='Code géographique_CODGEO', right_on = 'code_commune_INSEE')
dfcomgeo = dfcomgeo.drop(["nom_commune_postal", "nom_commune"], axis=1)

#Fusion commune, lycée 
dfmodel = pd.merge(dflyc, dfcomgeo, how = 'outer', left_on = "Code_commune", right_on ='code_commune_INSEE')

In [70]:
dfmodel['Libellé géographique_LIBGEO'] = dfmodel['Libellé géographique_LIBGEO'].str.upper()
dfmodelc = dfmodel.copy()


dfmodelc["Informations établissement_Ville"]= dfmodelc["Informations établissement_Ville"].fillna(0)
dfmodelcnz = dfmodelc[dfmodelc["Informations établissement_Ville"]!=0]
dfmodelcz = dfmodelc[dfmodelc["Informations établissement_Ville"]==0]
for i in range(6800):
    dfmodelcnz.loc[dfmodelcnz.index[i], "Libellé géographique_LIBGEO"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Informations établissement_Ville"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "code_commune_INSEE"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Code_commune"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "Médiane (€)_Q219_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Médiane (€)_Q219_x"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "latitude_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"latitude_x"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "longitude_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"longitude_x"]
dfmodelc = pd.concat([dfmodelcnz, dfmodelcz])
dfmodelc = dfmodelc.drop(["Code_commune","Informations établissement_Ville","code_commune", "latitude_x", "longitude_x", "Médiane (€)_Q219_x", "Code géographique_CODGEO"], axis=1)
dfmodelc

IndexError: index 4209 is out of bounds for axis 0 with size 4209

In [44]:
'''dfmodelc["Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"] = dfmodelc["Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"].fillna(0)

for i in range(4209):
    if dfmodelc.loc[dfmodel.index[i], "Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"]==0:
        a = dfmodelc.loc[dfmodelc.index[i], 'code_commune_INSEE']
        j=0
        while df_com.loc[dfmodelc.index[j], 'Code géographique_CODGEO']!=a:
            dfmodelc.loc[dfmodelc.index[i],  "Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"] = df_com.loc[dfmodelc.index[j],  "Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"] 
            j = j+1'''
               


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat